In [1]:
import sys
assert sys.version_info >= (3, 5)
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import time
K = keras.backend
import pandas as pd
import math
import os
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import uniform,randint
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from scipy.stats import norm



tf.random.set_seed(42)
np.random.seed(42)
random_state=42

save_path = 'save_path'
os.chdir(save_path)

X_train_omics_labelled = pd.read_csv("X_train_omics_labelled.csv",index_col=0)
X_test_omics= pd.read_csv("X_test_omics.csv",index_col=0)
X_valid_omics= pd.read_csv("X_valid_omics.csv",index_col=0)
features = np.load("feature_selection.npy",allow_pickle=True)

train_set_labelled_y= pd.read_csv("train_set_labelled_y.csv",index_col=0)
test_set_labelled_y= pd.read_csv("test_set_labelled_y.csv",index_col=0)
valid_set_labelled_y= pd.read_csv("valid_set_labelled_y.csv",index_col=0)

X_train_omics_labelled = X_train_omics_labelled[features]
X_test_omics = X_test_omics[features]
X_valid_omics = X_valid_omics[features]

train_set_labelled_c= pd.read_csv("train_set_labelled_c.csv",index_col=0)
test_set_labelled_c= pd.read_csv("test_set_labelled_c.csv",index_col=0)
valid_set_labelled_c= pd.read_csv("valid_set_labelled_c.csv",index_col=0)

#bin y 
from sklearn.preprocessing import KBinsDiscretizer
binner = KBinsDiscretizer(n_bins=10,encode="onehot-dense",strategy="uniform")
train_set_labelled_y = binner.fit_transform(train_set_labelled_y)
valid_set_labelled_y = binner.transform(valid_set_labelled_y)
test_set_labelled_y = binner.transform(test_set_labelled_y)

In [2]:
input_shape = X_train_omics_labelled.shape[1]

save_model_path = 'save_model_path'
os.chdir(save_model_path)

# Custom parts # 

## Helpful functions ## 

In [3]:
def validation_log_lik_sampling(y_val,x_val,variational_decoder,codings_size,samples=200):
    """
    Samples a value of z for the expectation, and calculates something proportional to loglikelihood.
    
    The more samples of z, the better the MC approximation to loglik.
    
    This is how we do our evaluation on the validation and also test set. 
    
    We look at the ability to generate x given y i.e. loglik(x|y)"""
    
    x_val_len = len(x_val)
    expectation = 0
    for i in range(samples):
        z = np.random.normal(loc=0,scale=1,size=codings_size*x_val_len).reshape(x_val_len,codings_size)
        x_pred = variational_decoder([z,y_val])
        diff = (x_val-x_pred)**2
        pdf = K.sum(diff,axis=-1)
        pdf = K.exp(-pdf)
        expectation += pdf 
    expectation = expectation / samples
    lik = tf.math.log(expectation)
    lik = K.mean(lik)    
    return lik



def create_batch_label(x_label, y_label, batch_s=32):
    '''
    Creates batches of labelled and unlabelled data. The total number of points in both batches is equal to batch_s. 
    
    Thanks to Omer for help
    '''
    proportion_labelled = x_label.shape[0]/(x_label.shape[0])
    
    shape_label = x_label.shape[0]
    label_per_batch = int(np.ceil(proportion_labelled*batch_s))
    batch_idx_la = np.random.choice(list(range(shape_label)), label_per_batch)
    batch_x_la = (x_label.iloc[batch_idx_la, :])
    batch_y_la = (y_label[batch_idx_la,:])
    
    del batch_idx_la
            
    return batch_x_la, batch_y_la


def progress_bar(iteration, total, size=30):
    """Progress bar for training"""
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    """Status bar for training"""
    metrics = " - ".join(["Loss for batch: {:.4f}".format(loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
def print_status_bar_epoch(iteration, total, training_loss_for_epoch,val_loss, metrics=None, size=30):
    """Status bar for training (end of epoch)"""
    metrics = " - ".join(
        ["trainLoss: {:.4f}  Val_loss: {:.4f} ".format(
            training_loss_for_epoch,val_loss)]
    )
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
    
def list_average(list_of_loss):
    return sum(list_of_loss)/len(list_of_loss)


def y_pred_loss(y_in):
    """Calculates loss and true y distribution given some y data.
    
    When the model calculates this it does it in batches (unlike this function which can take the whole data in).
    
    Therefore the model's learned distribution will probably not be as good as what is learnt when using the whole
    dataset. But that is one of the things that happens if we use mini-batch gradient descent.
    
    """
    y_distribution = (K.sum(y_in,axis=0) / len(y_in))
    loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y_in,y_distribution))
    return loss,y_distribution 


def rounded_accuracy(y_true,y_pred):
    """
    Calculates accuracy of classification predictions.
    
    For the 10D p vector which is y_pred, it sets the highest number to 1 and the rest to 0.
    
    It then computes accuracy.
    """
    from sklearn.metrics import accuracy_score
    b = np.zeros_like(y_pred)
    b[np.arange(len(y_pred)),y_pred.argmax(1)] = 1
    return accuracy_score(y_true,b)

## Model ##

### Custom components ###

In [4]:
class Sampling(keras.layers.Layer):
    """reparameterization trick"""
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean
    
class y_dist(keras.layers.Layer):
    """
    Custom layer that is used to learn the parameters, p, of the distribution over y.
    
    Outputs a loss and p. The loss is used for training. The loss is the categorical cross entropy loss between 
    p and every y sample. The mean of this is then taken to provide a per batch loss. 
    
    Shapes are configured for a 10D y.
    
    """
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        
    def build(self,batch_input_shape):
        self.q = self.add_weight(name="q",shape=[1,9],initializer="uniform",trainable=True)
        super().build(batch_input_shape)
    
    def call(self,X):
        concatenated = tf.concat([self.q,tf.constant(np.array(0.0).reshape(1,-1),dtype="float32")],axis=-1)
        p = K.exp(concatenated)
        p = tf.math.divide(p,K.sum(p))
        loss = keras.losses.categorical_crossentropy(X,p)
        loss = tf.reduce_mean(loss)
        return loss,p 
    
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(10)
    

class FullModel_only_labelled(keras.models.Model):
    def __init__(self,N_parameter,beta,variational_encoder,variational_decoder,classifier,y_distribution,**kwargs):
        super().__init__(**kwargs)
        self.encoder = variational_encoder
        self.decoder = variational_decoder
        self.classifier = classifier  
        self.y_distribution = y_distribution
        self.N = N_parameter
        self.beta = beta
    def call(self,inputs):
        """Inputs is a list, as such:
            inputs[0] is labelled X 
            inputs[1] is labelled y 
            """
        
        X_labelled = inputs[0]
        y_labelled = inputs[1]
        
        ############### LABELLED CASE #################
        
        codings_mean,codings_log_var,codings = self.encoder([X_labelled,y_labelled])
        y_pred_label = self.classifier(X_labelled)
        reconstructions = self.decoder([codings,y_labelled])

        #LOSSES#
        
        recon_loss = labelled_loss_reconstruction(codings_log_var,codings_mean,X_labelled,reconstructions,self.beta)
        cls_loss = labelled_cls_loss(y_labelled,y_pred_label,self.N)
        y_dist_loss = self.y_distribution(y_labelled)[0]
        labelled_loss = recon_loss + cls_loss + y_dist_loss
       
        
        ############### ALL LOSSES #######################
        
        loss = labelled_loss
        return loss

    
def build_model_only_labelled(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50):
    
    """
    Builds deep generative model.
    
    Parameters specify the architecture. Architecture is such that encoder and decoder have same number of nodes and hidden
    layers. Done for simplicity. Classifier has its own architecture.
    
    Returns encoder,decoder,y_distribution, classifier and overall model. These can be used downstream.
    
    """
       
    ########## ENCODER ###############
    
    x_in = keras.layers.Input(shape=[input_shape])
    y_in = keras.layers.Input(shape=[10])
    z = keras.layers.concatenate([x_in,y_in])
    for layer in range(n_hidden):
        z = keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal")(z)
        z = keras.layers.Dropout(0.3)(z)

    codings_mean = keras.layers.Dense(codings_size)(z)
    codings_log_var = keras.layers.Dense(codings_size)(z)
    codings = Sampling()([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(
        inputs=[x_in,y_in], outputs=[codings_mean, codings_log_var, codings])
    
    
    ########## DECODER ###############

    latent = keras.layers.Input(shape=[codings_size])
    l_merged = keras.layers.concatenate([latent,y_in])
    x = l_merged
    for layer in range(n_hidden):
        x = keras.layers.Dense(n_neurons, activation="elu",kernel_initializer="he_normal")(x)
        x = keras.layers.Dropout(0.3)(x)
    x_out = keras.layers.Dense(input_shape,activation="sigmoid")(x) 
    variational_decoder = keras.models.Model(inputs=[latent,y_in], outputs=[x_out])
    
    
    ########### CLASSIFIER ############
    
    y_classifier = x_in
    for layer in range(n_hidden_classifier):
        y_classifier = keras.layers.Dense(n_neurons_classifier, activation="elu",kernel_initializer="he_normal")(y_classifier)
        y_classifier = keras.layers.Dropout(rate=0.3)(y_classifier)
    y_pred = keras.layers.Dense(10,activation="softmax")(y_classifier) 
    classifier = keras.models.Model(inputs=[x_in], outputs=[y_pred])
    
    
    ############ Y DISTRIBUTION #############
    
    loss,p = y_dist()(y_in)
    y_distribution = keras.models.Model(inputs=[y_in],outputs=[loss,p])
    
    
    ########## FULL MODEL #############
    
    model = FullModel_only_labelled(N_parameter=N,beta=beta,variational_encoder=variational_encoder,
                  variational_decoder=variational_decoder,classifier=classifier,y_distribution=y_distribution)
    
    return variational_encoder,variational_decoder,classifier,y_distribution,model

### Loss functions ###

In [5]:
def custom_mse(x,x_decoded_mean):
    """returns column of squared errors. Length of column is number of samples."""
    diff = (x-x_decoded_mean)**2
    return K.sum(diff,axis=-1) /2 


def labelled_loss_reconstruction(codings_log_var,codings_mean,x, x_decoded_mean,beta=1):
    """Labelled data. This is the reconstruction loss for the x."""
    recon_loss = custom_mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
    #kl loss gives vector of one value for each sample in batch 
    return K.mean(recon_loss + beta*kl_loss)


def dummy_loss(y,ypred):
    """This is a dummy loss that returns a value of zero. It is here as keras requires a loss term for each output.
        The regression_loss_for_labelled_y gives the loss which depends on the log var and mean, so we don't need another
        loss. But keras wants us to give separate losses for each. To keep keras happy, we use the dummy loss as a placeholder."""
    return 0.0

def labelled_cls_loss(y, y_pred,N=383):
    alpha = 0.1*N
    cat_xent_loss = keras.losses.categorical_crossentropy(y, y_pred)
    return alpha*K.mean(cat_xent_loss)

### Training functions ###

In [6]:
@tf.function
def train_step(inputs):
    with tf.GradientTape() as tape:
        loss = model(inputs,training=True)
        loss = tf.add_n([loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def fit_model(X_train_la, y_train_la,epochs,X_valid_la, y_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,
             valid_set=True):

    """
    Fits a single model. Gets the validation loss too if valid set exists. 
    And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Valid set is True or False depending if you have one. If you don't, the model at the end of training is saved.
    You must still pass in dummy valid sets even if valid_set=False.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    e.g. usage fit_model(X_train_omics_labelled, train_set_labelled_y,50,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,valid_set=True)
    """
    if valid_set is True:
    
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        validation_loss = []
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))

                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la= create_batch_label(
                        X_train_la, y_train_la,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),variational_decoder=variational_decoder,codings_size=codings_size)

                validation_loss.append(val_loss)
                print_status_bar_epoch(X_train_la.shape[0] 
                                 ,(X_train_la.shape[0] ),training_loss_for_epoch,val_loss )

                #callback for early stopping
                if epoch <= patience - 1:

                    if epoch == 0:

                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")

                    else:
                        if all(val_loss<i for i in validation_loss[:-1]) is True:
                            variational_encoder.save("variational_encoder.h5")
                            variational_decoder.save("variational_decoder.h5")
                            classifier.save("classifier.h5")
                            y_distribution.save("y_distribution.h5")
                #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
                #then the statement below won't save any model, which is undesirable as we need to load a model. 

                if epoch > patience - 1:

                    latest_val_loss = validation_loss[-patience:]
                    if all(val_loss<i for i in latest_val_loss[:-2]) is True:
                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")
                    if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                        break     

        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5")     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        print("best val loss: ", min(validation_loss))
        
        return history, min(validation_loss)

        
    else:
        
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))        
        val_loss = 0

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))
                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la= create_batch_label(
                        X_train_la, y_train_la,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                print_status_bar_epoch(X_train_la.shape[0]
                                 ,(X_train_la.shape[0]),training_loss_for_epoch,val_loss )
        

        variational_encoder.save("variational_encoder.h5")
        variational_decoder.save("variational_decoder.h5")
        classifier.save("classifier.h5")
        y_distribution.save("y_distribution.h5")
        
        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5")     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        
    
        return history
    
def fit_model_search_labelled(X_train_la, y_train_la,epochs,X_valid_la, y_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,
                    codings_size=50):

    """
    Use for hyperparameter search. 
    
    Fits the model. Gets the validation loss too. And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    """
    
    start = time.time()
    history = []
    
        
    @tf.function
    def train_step(inputs):
        with tf.GradientTape() as tape:
            loss = model(inputs,training=True)
            loss = tf.add_n([loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    validation_loss = []
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)

    batch_loss = []
    
    batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))

    for epoch in range(epochs):
            
            print("Epoch {}/{}".format(epoch,epochs))
            
            for i in range(batches_per_epoch):
                
                batch_x_la, batch_y_la= create_batch_label(
                    X_train_la, y_train_la,batch_size)

                inputs = [batch_x_la.to_numpy(),batch_y_la]
                loss = train_step(inputs)

                batch_loss.append(loss)
                
                average_batch_loss = list_average(batch_loss)
                
                print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)
            
            training_loss_for_epoch = list_average(batch_loss)

            batch_loss = []
                
            history.append(training_loss_for_epoch)
            
            val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),variational_decoder=variational_decoder,codings_size=codings_size)
            
            validation_loss.append(val_loss)
            
            print_status_bar_epoch(X_train_la.shape[0] 
                             ,(X_train_la.shape[0] ),training_loss_for_epoch,val_loss )
            
            #callback for early stopping
            
            if epoch <= patience - 1:
                
                if epoch == 0:
                
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                    
                else:
                    if all(val_loss<i for i in validation_loss[:-1]) is True:
                        variational_encoder.save("variational_encoder_intermediate.h5")
                        variational_decoder.save("variational_decoder_intermediate.h5")
                        classifier.save("classifier_intermediate.h5")
                        y_distribution.save("y_distribution_intermediate.h5")
            #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
            #then the statement below won't save any model, which is undesirable as we need to load a model. 
            
            if epoch > patience - 1:
                                
                latest_val_loss = validation_loss[-patience:]
                if all(val_loss<i for i in latest_val_loss[:-1]) is True:
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                    break     
    
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder_intermediate.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder_intermediate.h5")
    classifier = keras.models.load_model("classifier_intermediate.h5")     
    y_distribution = keras.models.load_model("y_distribution_intermediate.h5", custom_objects={
       "y_dist": y_dist
    })    
                
    done = time.time()
    elapsed = done-start
    print("Elapsed/s: ",elapsed)
    print("Final training loss: ",training_loss_for_epoch)
    print("best val loss: ", min(validation_loss))
    
    return history, min(validation_loss)


def hyperparameter_search_labelled(param_distribs,epochs,patience,n_iter,X_train_la=X_train_omics_labelled, 
                          y_train_la=train_set_labelled_y,
                          X_valid_la=X_valid_omics, y_valid_la=valid_set_labelled_y):
    
    """
    Performs hyperparameter, random search. Assesses performance by determining the score on the validation set. 
    
    Saves best models (encoder, decoder and regressor) and returns these. These can then be used downstream.
    
    Maybe want to implement something where the best model is then retrained on train+valid set?
    
    Param_distribs of the form: 
            param_distribs = {
            "n_hidden": [1],
            "n_hidden_classifier": [1],
            "beta": [1],
            "n_neurons": randint.rvs(50,1000-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(49,1000-49,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(50,290-50,size=30,random_state=random_state).tolist(),
            "N" :randint.rvs().tolist(),
            "learning_rate" : ....
            #"codings_size": [50]}
            
    There must be a value for every parameter. If you know the value you want to use, set it in the param_distribs
    dictionary.
    
    Patience must be less than the number of epochs.
    
    e.g. result,variational_encoder,variational_decoder,classifier,y_distribution =
            hyperparameter_search(param_distribs,500,10,n_iter=10)

            
    """

    np.random.seed(42) #needs to be here so that everything that follows is consistent

    min_val_loss = []
    master = {}

    for i in range(n_iter): 
        K.clear_session()
        master[i] = {}
        master[i]["parameters"] = {}
        
        N= np.random.choice(param_distribs["N"])
        learning_rate= np.random.choice(param_distribs["learning_rate"])
        beta= np.random.choice(param_distribs["beta"])
        n_neurons =np.random.choice(param_distribs["n_neurons"]) 
        n_neurons_classifier =np.random.choice(param_distribs["n_neurons_classifier"]) 
        n_hidden  =np.random.choice(param_distribs["n_hidden"]) 
        n_hidden_classifier  =np.random.choice(param_distribs["n_hidden_classifier"]) 
        codings_size =np.random.choice(param_distribs["codings_size"]) 
       
        master[i]["parameters"]["N"] = N
        master[i]["parameters"]["learning_rate"] = learning_rate
        master[i]["parameters"]["beta"] = beta
        master[i]["parameters"]["n_neurons"] = n_neurons
        master[i]["parameters"]["n_neurons_classifier"] = n_neurons_classifier
        master[i]["parameters"]["n_hidden"] = n_hidden
        master[i]["parameters"]["n_hidden_classifier"] = n_hidden_classifier
        master[i]["parameters"]["codings_size"] = codings_size

        
        variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_only_labelled(n_hidden=n_hidden,       
                                       n_neurons=n_neurons,beta=beta,n_hidden_classifier=n_hidden_classifier,
                                        n_neurons_classifier=n_neurons_classifier,N=N,codings_size=codings_size)
        
                
        history,val_loss = fit_model_search_labelled(X_train_la=X_train_la, y_train_la=y_train_la, 
                                 epochs=epochs,X_valid_la=X_valid_la, 
                                 y_valid_la=y_valid_la,patience=patience,variational_encoder=variational_encoder,
                                variational_decoder=variational_decoder, classifier=classifier,
                                y_distribution=y_distribution,model=model,Sampling=Sampling,y_dist=y_dist,
                                            batch_size=32,learning_rate=learning_rate,codings_size=codings_size)
        
        

        master[i]["val_loss"] = val_loss
        min_val_loss.append(val_loss)

        #If val loss is lowest, save this model. 
        if val_loss <=  min(min_val_loss):
            os.rename("variational_encoder_intermediate.h5","variational_encoder.h5")
            os.rename("variational_decoder_intermediate.h5","variational_decoder.h5")
            os.rename("classifier_intermediate.h5","classifier.h5")
            os.rename("y_distribution_intermediate.h5","y_distribution.h5")

        print(master)
            
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder.h5")
    classifier = keras.models.load_model("classifier.h5")     
    y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
       "y_dist": y_dist
    })    

    result = sorted(master.items(), key=lambda item: item[1]["val_loss"])
    return result,variational_encoder,variational_decoder,classifier,y_distribution


# Hyperparameter Search #

In [8]:
param_distribs = {
            "n_hidden": [1,2],
            "n_hidden_classifier": [1,2],
            "beta": [1,10,15],
            "n_neurons": randint.rvs(50,600-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(20,120-20,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(20,290-20,size=30,random_state=random_state).tolist(),
            "N" :[1,50,100,150],
            "learning_rate" : [0.001,0.0005,0.0001],
            #"codings_size": [120,60]
}

In [9]:
result,variational_encoder,variational_decoder,classifier,y_distribution = hyperparameter_search_labelled(param_distribs,500,10,n_iter=10)
result

Epoch 0/500

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

320/383 [========================>.....] - Loss for batch: 523.9136WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change a

383/383 [==============================] - trainLoss: 13.0800  Val_loss: 12.7978 
Epoch 56/500
383/383 [==============================] - trainLoss: 12.9159  Val_loss: 12.8688 
Epoch 57/500
383/383 [==============================] - trainLoss: 12.7190  Val_loss: 12.8232 
Epoch 58/500
383/383 [==============================] - trainLoss: 13.0620  Val_loss: 12.8096 
Epoch 59/500
383/383 [==============================] - trainLoss: 12.7859  Val_loss: 12.8139 
Epoch 60/500
383/383 [==============================] - trainLoss: 12.7502  Val_loss: 12.8181 
Epoch 61/500
383/383 [==============================] - trainLoss: 12.8353  Val_loss: 12.8021 
Epoch 62/500
383/383 [==============================] - trainLoss: 13.1002  Val_loss: 12.8019 
Elapsed/s:  37.84289288520813
Final training loss:  tf.Tensor(13.100234, shape=(), dtype=float32)
best val loss:  tf.Tensor(12.758459, shape=(), dtype=float32)
{0: {'parameters': {'codings_size': 208, 'n_hidden_classifier': 1, 'N': 100, 'beta': 15, 'n_n

383/383 [==============================] - trainLoss: 39.3131  Val_loss: 14.5845 
Epoch 137/500
383/383 [==============================] - trainLoss: 39.6620  Val_loss: 14.5759 
Epoch 138/500
383/383 [==============================] - trainLoss: 39.0237  Val_loss: 14.5636 
Epoch 139/500
383/383 [==============================] - trainLoss: 38.2134  Val_loss: 14.5302 
Epoch 140/500
383/383 [==============================] - trainLoss: 38.2463  Val_loss: 14.4932 
Epoch 141/500
383/383 [==============================] - trainLoss: 38.9267  Val_loss: 14.4773 
Epoch 142/500
383/383 [==============================] - trainLoss: 38.4717  Val_loss: 14.4541 
Epoch 143/500
383/383 [==============================] - trainLoss: 37.1069  Val_loss: 14.4329 
Epoch 144/500
383/383 [==============================] - trainLoss: 36.9219  Val_loss: 14.4111 
Epoch 145/500
383/383 [==============================] - trainLoss: 37.6413  Val_loss: 14.4243 
Epoch 146/500
383/383 [==============================]

383/383 [==============================] - trainLoss: 20.9254  Val_loss: 13.0892 
Epoch 307/500
383/383 [==============================] - trainLoss: 20.2571  Val_loss: 13.0988 
Epoch 308/500
383/383 [==============================] - trainLoss: 20.5562  Val_loss: 13.0918 
Epoch 309/500
383/383 [==============================] - trainLoss: 20.2764  Val_loss: 13.0839 
Epoch 310/500
383/383 [==============================] - trainLoss: 19.9730  Val_loss: 13.0725 
Epoch 311/500
383/383 [==============================] - trainLoss: 20.1566  Val_loss: 13.0520 
Epoch 312/500
383/383 [==============================] - trainLoss: 20.3394  Val_loss: 13.0443 
Epoch 313/500
383/383 [==============================] - trainLoss: 20.5328  Val_loss: 13.0357 
Epoch 314/500
383/383 [==============================] - trainLoss: 20.1898  Val_loss: 13.0406 
Epoch 315/500
383/383 [==============================] - trainLoss: 20.3756  Val_loss: 13.0266 
Epoch 316/500
383/383 [==============================]

383/383 [==============================] - trainLoss: 13.2795  Val_loss: 13.1720 
Epoch 30/500
383/383 [==============================] - trainLoss: 12.5820  Val_loss: 13.2352 
Epoch 31/500
383/383 [==============================] - trainLoss: 12.6542  Val_loss: 13.2733 
Epoch 32/500
383/383 [==============================] - trainLoss: 12.3948  Val_loss: 13.3122 
Epoch 33/500
383/383 [==============================] - trainLoss: 12.1986  Val_loss: 13.1443 
Epoch 34/500
383/383 [==============================] - trainLoss: 11.8291  Val_loss: 13.0422 
Epoch 35/500
383/383 [==============================] - trainLoss: 11.4594  Val_loss: 13.0038 
Epoch 36/500
383/383 [==============================] - trainLoss: 11.4230  Val_loss: 12.9791 
Epoch 37/500
383/383 [==============================] - trainLoss: 11.1419  Val_loss: 13.1620 
Epoch 38/500
383/383 [==============================] - trainLoss: 11.1207  Val_loss: 13.1670 
Epoch 39/500
383/383 [==============================] - trainLo

...

{0: {'parameters': {'codings_size': 208, 'n_hidden_classifier': 1, 'N': 100, 'beta': 15, 'n_neurons': 516, 'n_neurons_classifier': 43, 'learning_rate': 0.001, 'n_hidden': 1}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.73137>}, 1: {'parameters': {'codings_size': 207, 'n_hidden_classifier': 1, 'N': 50, 'beta': 1, 'n_neurons': 152, 'n_neurons_classifier': 40, 'learning_rate': 0.001, 'n_hidden': 1}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.758459>}, 2: {'parameters': {'codings_size': 177, 'n_hidden_classifier': 2, 'N': 50, 'beta': 15, 'n_neurons': 398, 'n_neurons_classifier': 80, 'learning_rate': 0.0001, 'n_hidden': 1}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=13.014389>}, 3: {'parameters': {'codings_size': 57, 'n_hidden_classifier': 2, 'N': 1, 'beta': 15, 'n_neurons': 516, 'n_neurons_classifier': 43, 'learning_rate': 0.001, 'n_hidden': 1}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.97913>}, 4: {'parameters': {'codings_size': 126

383/383 [==============================] - trainLoss: 114.0568  Val_loss: 20.4388 
Epoch 15/500
383/383 [==============================] - trainLoss: 112.3851  Val_loss: 20.1855 
Epoch 16/500
383/383 [==============================] - trainLoss: 107.9404  Val_loss: 19.9331 
Epoch 17/500
383/383 [==============================] - trainLoss: 104.0274  Val_loss: 19.7083 
Epoch 18/500
383/383 [==============================] - trainLoss: 100.2401  Val_loss: 19.4536 
Epoch 19/500
383/383 [==============================] - trainLoss: 95.0149  Val_loss: 19.1851 
Epoch 20/500
383/383 [==============================] - trainLoss: 96.4373  Val_loss: 19.0020 
Epoch 21/500
383/383 [==============================] - trainLoss: 93.1035  Val_loss: 18.8187 
Epoch 22/500
383/383 [==============================] - trainLoss: 93.4867  Val_loss: 18.6814 
Epoch 23/500
383/383 [==============================] - trainLoss: 86.4179  Val_loss: 18.5058 
Epoch 24/500
383/383 [==============================] - tr

383/383 [==============================] - trainLoss: 20.0047  Val_loss: 13.4636 
Epoch 186/500
383/383 [==============================] - trainLoss: 20.5269  Val_loss: 13.4392 
Epoch 187/500
383/383 [==============================] - trainLoss: 20.1262  Val_loss: 13.4380 
Epoch 188/500
383/383 [==============================] - trainLoss: 19.5478  Val_loss: 13.4127 
Epoch 189/500
383/383 [==============================] - trainLoss: 19.8065  Val_loss: 13.3892 
Epoch 190/500
383/383 [==============================] - trainLoss: 19.6267  Val_loss: 13.3672 
Epoch 191/500
383/383 [==============================] - trainLoss: 19.7830  Val_loss: 13.3682 
Epoch 192/500
383/383 [==============================] - trainLoss: 19.3273  Val_loss: 13.3699 
Epoch 193/500
383/383 [==============================] - trainLoss: 19.2940  Val_loss: 13.3768 
Epoch 194/500
383/383 [==============================] - trainLoss: 19.6538  Val_loss: 13.3815 
Epoch 195/500
383/383 [==============================]

383/383 [==============================] - trainLoss: 30.2670  Val_loss: 14.6102 
Epoch 27/500
383/383 [==============================] - trainLoss: 29.5120  Val_loss: 14.5233 
Epoch 28/500
383/383 [==============================] - trainLoss: 29.3031  Val_loss: 14.4160 
Epoch 29/500
383/383 [==============================] - trainLoss: 28.2803  Val_loss: 14.3146 
Epoch 30/500
383/383 [==============================] - trainLoss: 27.9622  Val_loss: 14.2724 
Epoch 31/500
383/383 [==============================] - trainLoss: 27.2859  Val_loss: 14.2190 
Epoch 32/500
383/383 [==============================] - trainLoss: 27.6318  Val_loss: 14.1706 
Epoch 33/500
383/383 [==============================] - trainLoss: 26.6218  Val_loss: 14.1399 
Epoch 34/500
383/383 [==============================] - trainLoss: 26.2902  Val_loss: 14.0635 
Epoch 35/500
383/383 [==============================] - trainLoss: 26.3932  Val_loss: 14.0279 
Epoch 36/500
383/383 [==============================] - trainLo

[(0,
  {'parameters': {'N': 100,
    'beta': 15,
    'codings_size': 208,
    'learning_rate': 0.001,
    'n_hidden': 1,
    'n_hidden_classifier': 1,
    'n_neurons': 516,
    'n_neurons_classifier': 43},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.73137>}),
 (1,
  {'parameters': {'N': 50,
    'beta': 1,
    'codings_size': 207,
    'learning_rate': 0.001,
    'n_hidden': 1,
    'n_hidden_classifier': 1,
    'n_neurons': 152,
    'n_neurons_classifier': 40},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.758459>}),
 (5,
  {'parameters': {'N': 50,
    'beta': 1,
    'codings_size': 230,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 2,
    'n_neurons': 380,
    'n_neurons_classifier': 57},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.909143>}),
 (9,
  {'parameters': {'N': 50,
    'beta': 10,
    'codings_size': 255,
    'learning_rate': 0.0005,
    'n_hidden': 1,
    'n_hidden_classifier': 1,
    'n_neurons': 15

Best KL-based model has val nloglik of 12.73137. Epochs = 66

{'parameters': {'N': 100,
    'beta': 15,
    'codings_size': 208,
    'learning_rate': 0.001,
    'n_hidden': 1,
    'n_hidden_classifier': 1,
    'n_neurons': 516,
    'n_neurons_classifier': 43},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.73137>}
     

# Single run # 

In [8]:
variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_only_labelled(n_hidden=1, 
                    n_neurons=516,input_shape=input_shape,beta=15,n_hidden_classifier=1,
              n_neurons_classifier=43,N=100,codings_size=208)

In [9]:
fit_model(X_train_omics_labelled, train_set_labelled_y,150,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=208,valid_set=True)

Epoch 0/150

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

320/383 [========================>.....] - Loss for batch: 611.4309WARNING:tensorflow:Layer concatenate_3 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change a

([<tf.Tensor: shape=(), dtype=float32, numpy=611.4309>,
  <tf.Tensor: shape=(), dtype=float32, numpy=193.40825>,
  <tf.Tensor: shape=(), dtype=float32, numpy=135.0973>,
  <tf.Tensor: shape=(), dtype=float32, numpy=116.42486>,
  <tf.Tensor: shape=(), dtype=float32, numpy=103.136566>,
  <tf.Tensor: shape=(), dtype=float32, numpy=97.52397>,
  <tf.Tensor: shape=(), dtype=float32, numpy=91.77984>,
  <tf.Tensor: shape=(), dtype=float32, numpy=84.663>,
  <tf.Tensor: shape=(), dtype=float32, numpy=80.755394>,
  <tf.Tensor: shape=(), dtype=float32, numpy=78.03452>,
  <tf.Tensor: shape=(), dtype=float32, numpy=76.26581>,
  <tf.Tensor: shape=(), dtype=float32, numpy=71.808075>,
  <tf.Tensor: shape=(), dtype=float32, numpy=69.796135>,
  <tf.Tensor: shape=(), dtype=float32, numpy=66.47869>,
  <tf.Tensor: shape=(), dtype=float32, numpy=64.37159>,
  <tf.Tensor: shape=(), dtype=float32, numpy=63.35486>,
  <tf.Tensor: shape=(), dtype=float32, numpy=61.64589>,
  <tf.Tensor: shape=(), dtype=float32, nump

In [10]:
test_nlog_lik = -validation_log_lik_sampling(test_set_labelled_y,X_test_omics.to_numpy(),
                                    variational_decoder=variational_decoder,codings_size=208,samples=2000)
print("test_nlog_lik = " + str(test_nlog_lik))

test_nlog_lik = tf.Tensor(13.166075, shape=(), dtype=float32)
